<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="45%"><img src="../media/Univ-Utah.jpeg"><br>
</td>
    <td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">University of Utah<br>Population Health Sciences<br>Data Science Workshop</font></h1></td>
<td valign="center" align="center" width="45%"><img
src="../media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


In [ ]:
from helpers import *
import pandas as pd

# Introduction to SQL
Now that we have some background about what databases are and how they're structured, we'll get some hands-on experiencing joining tables and querying data from MIMIC.

## Connecting to MIMIC
Throughout this class, we'll use the function `connect_to_mimic` to connect to the MIMIC database (imported as part of the `helpers` module). This requires the package `pymysql`, so you may have to install that first. 


#### TODO
In the cells below, install `pymysql` and then run the function to connect to the database. Ask your instructor for the password when prompted.

In [ ]:
!pip install pymysql

In [ ]:
conn = connect_to_mimic("uu-phs")
conn

Great, we've connected to MIMIC! Now we're almost ready to pull some data. But first we need to learn some basic SQL.

## Administrative and demographic data
For next couple of notebooks, we'll focus on tables containing **ddministrative and demographic data**. These tables define general information about the patient or their hospitalizations. This includes data elements such as:
- Name
- Sex
- Date of birth
- Insurance information
- Admit/discharge datetime

We'll focus on three tables for now: `admission`, `d_patients`, and `demographic_detail`. Let's start writing some queries in SQL!

## SQL
**Structured Query Language (SQL)** is a programming language used to interact with many relational databases. There are many different *flavors* of SQL that vary slightly from one another, but the core logic is typically the same.

When we use SQL, we execute a **query** which runs some logic to specifyl, filter, and transform data in the database. It then returns the **result set** to us. In Python, we can use the `pandas` function `read_sql` to connect to a database and execute a query.  `pd.read_sql` takes two required arguments:

- `sql`: A string containing a SQL query
- `con`: The connection object which allows us to access the database 
For the second argument, we'll use the `conn` object returned by `connect_to_mimic`. The first argument should be a string containing SQL code.

Here is an example of executing a query:

In [ ]:
query = """
SELECT *
FROM admissions
WHERE admit_dt <= '3033-07-08 00:00:00'
LIMIT 10;
"""

pd.read_sql(query, conn)

## Structure of a SQL query

Let's go back through that SQL query. If we were to translate the query to natural language, we might express it as:

---
**"Give me the top 10 rows of data from the `admissions` table where the admit datetime was before July 8th, 3033."** (Why do you think the dates look so weird?)

---

This returns a pandas DataFrame with four columns and 10 rows.

#### Discussion
What columns are returned by this query? What do the columns/values mean?

</br>
More generally, here is the structure of a SQL query:

--- 
<html>
<font>
    <p style="font-family:courier";>SELECT COLUMN NAMES (or *)</p>
    <p style="font-family:courier";> FROM TABLE NAME</p>
    <p style="font-family:courier";>(optional) JOIN</p>
    <p style="font-family:courier";>(optional) WHERE (condition)</p>
    <p style="font-family:courier";>(optional) ORDER BY ...</p>
    <p style="font-family:courier";>(optional) LIMIT N ...</p>
</html>
---

Here is a quick explanation of each clause (we'll go through each in detail later):
- `SELECT`: This tells us which columns we want to pull. If we say `SELECT *`, that means `"SELECT ALL"`
- `FROM`: This specifies which table the data will be in
- `JOIN`: This joins two tables together using a common key. If we only need a single table, we can leave this out.
- `WHERE`: This allows to filter to where rows where a certain condition is matched
- `ORDER BY`: This sorts the rows by a particular column
- `LIMIT`: This means we only want the first `N` rows. In this class, we'll typically use this clause so we don't pull excessively large datasets.

    
#### TODO

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_simple_query_parts

Here is a slightly more complicated query:

In [ ]:
query = """
SELECT a.hadm_id, a.subject_id, a.admit_dt, a.disch_dt, p.sex, p.dob, p.dod, p.hospital_expire_flg
FROM d_patients  p
    INNER JOIN admissions a
        ON p.subject_id = a.subject_id
WHERE hospital_expire_flg = 'Y'
ORDER BY dod
LIMIT 10;
"""

pd.read_sql(query, conn)

#### TODO

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_tables_in_query

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_order_by_column

## Writing queries
Now let's go through the different parts of a SQL query and get some practice writing our own queries.

### The essentials

There are two components that all of our SQL queries will have:
1. **The `SELECT` statement**: Here, we *select* the columns that we want to retrieve from the database. You can either list the specific columns you want separated or commas or just say **"*"** to pull all of them
2. **The `FROM` statement**: We need to specify what table these columns are coming from. We sometimes give a table name an "alias" (often a single letter) to refer to in the query.

While it's not always essential, in this class we will also often have:

3. **A `LIMIT`** statement**: Limit the number of rows we're pulling so we don't overwhelm the database (or your machine).

So a very basic query could just select all (or some) of the rows and all of the columns from a single table. The following query pulls every column from the first 10 rows of **d_patients** while giving the table an alias of `d`:  

In [ ]:
query = """
SELECT *
FROM d_patients d
LIMIT 10;
"""

pd.read_sql(query, conn)

#### TODO
Write and execute a query to select just the `hadm_id`, `subject_id`, `admit_dt`, and `disch_dt` from the `admissions` table. Give the table an alias of `a`. Limit to the first 25 rows. Save the result as `df`.

In [ ]:
query = """
SELECT hadm_id, ____, admit_dt, disch_dt
____ admissions a -- alias; using 'AS a' is optional
LIMIT ____;
"""

pd.read_sql(query, conn)

In [ ]:
test_query_result1.test(df)

## Joining tables
In a relational database like MIMIC, different attributes for entities are stored in different tables. These disparate tables can then be joined together in a query using a `join` statement. The column `subject_id`, which is the identifier for a patient, is consistent between these two columns and can be used to join them together:

```SQL
FROM table1
    INNER JOIN table2
        ON table1.column = table2.column
```

*Note*: We'll talk more about the phrase `INNER JOIN` later, as well as other types of joins.

#### TODO
Join the `demographic_detail` and `d_patients` tables using the `subject_id` column in both as the joining keys. Select all columns and the **top 10** rows.

In [ ]:
query = """
SELECT ____    
FROM ____ d
    INNER JOIN d_patients __
        ON d.____ = p.____
"""

In [ ]:
pd.read_sql(query, conn)

## Filtering results
Typically we don't want to return *all* rows from a table. We instead usually filter based on conditions related to the columns of the table. This is where the `WHERE` clause comes in.

For example, to get the demographic details for a single patient, we can filter based on the `subject_id` column:

In [ ]:
query = """
SELECT *
FROM demographic_detail d
WHERE subject_id = 78
"""
pd.read_sql(query, conn)

You can also use standard comparators like `!=`, `>`, `>=`, `<`, and `<=`. 

Run the query below - it returns an error. Scroll to the bottom of the error traceback and read the error description:

`"Column 'subject_id' in where clause is ambiguous"`


In [ ]:
query = """
SELECT p.subject_id, p.dob, p.dod, d.admission_type_descr
FROM demographic_detail d INNER JOIN
    d_patients p
        ON d.subject_id = p.subject_id
WHERE subject_id = 78
"""
pd.read_sql(query, conn)

#### TODO

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_error_ambiguous 

When more than one table in our query has a column of the same name, we need to specify which table we're referring to. We can specify a column using the notation:

`table_name.column_name`

or, if we're using aliases::

`alias.column_name`

For example:

`WHERE d.subject_id = 78`

or:

`WHERE p.subject_id = 78`

#### TODO
Which of the following changes to the `WHERE` clause would cause our query to run correctly?
- **a)** `WHERE d.subject_id = 78`
- **b)** `WHERE p.subject_id = 78`
- **c)** `WHERE ANY(subject_id) = 78`

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_fix_where_ambiguity

#### TODO
Based on your answer to the previous quiz, fix the query and rerun it, saving the result as `df`.

In [ ]:
query = """

"""
df = pd.read_sql(query, conn)
df

In [ ]:
# RUN CELL TO TEST VALUE
test_fixed_where_ambiguity.test(df)

## Ordering results
Finally, we can order the queried data by using the `ORDER BY` clause:

In [ ]:
query = """
SELECT *
FROM d_patients
WHERE subject_id IN (56, 78, 37)
ORDER BY subject_id;
"""
pd.read_sql(query, conn)

By default, `ORDER BY` sorts values in **ascending** order. But we can switch to **descending** order using the `DESC` keyword:

```sql
ORDER BY column DESC
```

#### TODO
Change the query above to sort the data by `dob` in *descending* order. Rerun and save as `df`.

In [ ]:
query = """

"""
df = pd.read_sql(query, conn)
df

In [ ]:
# RUN CELL TO TEST VALUE
test_query_dob_descending.test(df)

## Renaming columns
Sometimes we might want to rename our columns, maybe to make it a name that's easier to understand or that is less ambiguous. We do this the same way we assigned *aliases* to tables:

```sql
SELECT column1 AS new_name
    ,column2 new_name2 -- 'AS' is optional
```

#### TODO
Select the first 10 rows of `d_patients` and rename `dob` to `date_of_birth` and `dod` to `date_of_death`. Save the assignment as `df_patients_renamed`

In [ ]:
query = """
SELECT
    subject_id
    ,sex
    ,__ AS ____
    ,____
    ,hospital_expire_flg
FROM d_patients
LIMIT 10
"""

df_patients_renamed = pd.____(____, conn)
df_patients_renamed

In [ ]:
# RUN CELL TO TEST VALUE
validate_df_patients_renamed.test(df_patients_renamed)

## Creating new columns with new values
Often, tables don't have the exist data element we want. For example, let's say that we want to study patient age at death. There is no exact column for this in `d_patients`, but we can use `dod` and `dob` to calculate a new column.

Just like Python, SQL has certain **functions** that you can add to your queries. One such function is `DATEDIFF` which calculates the number of days between two dates:

```sql
SELECT DATEDIFF(date1, date2)
```

We can use that to calculate the number of days between patients' death and birth dates. When we calculate a new column we need to give it a name, so we'll call this one `age_at_death_days`.

In [ ]:
query = """
SELECT 
    subject_id,
    dod,
    DATEDIFF(dod, dob) age_at_death_days
FROM d_patients p
LIMIT 10
"""
pd.read_sql(query, conn)

We can also do basic arithmetic like addition, subtraction, multiplication, and division using operators similar to Python: `+`, `-`, `*`, `/`.

#### TODO
Make a new version of `df_patients` with all 4,000 rows and a new column called `age_at_death` which is the patient's age when they died *in years*.

In [ ]:
# RUN CELL TO SEE HINT
hint_age_in_years

In [ ]:
query = """
SELECT *
    ,____
"""

df_patients = pd.read_sql(query, conn)
df_patients.head()

In [ ]:
# RUN CELL TO TEST VALUE
test_age_at_death.test(df_patients)

## Aggregating data
So far, everything we've done with SQL has been at the **row-level**. That is, we've written queries that have returned results with a single entity (patient, hospitalization, etc.) per row. Next we'll start looking at how to **aggregate** data in SQL.

Some examples of aggregate data we coud compute in MIMIC include:
- The number of admissions
- The count of patients by sex
- The min and max admission date for patients
- The mean/min/max/standard deviation of length of stay

In `module_2`, we learned how to do many of these calculatins in Python using `pandas`. Aggregating data in SQL is very similar. Each of the calculations described above can be computed using a **SQL function** like `COUNT()`, `MIN()`, or `MAX()`.

### Counts
One of the most basic aggegations is simply counting the number of rows in a table. We can get this by selecting `COUNT(*)`:

In [ ]:
query = """
SELECT COUNT(1) AS n
FROM admissions
"""
pd.read_sql(query, conn)

#### TODO
How would you interpret the result above?

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_count_n

Aggregate queries can have other clauses like `WHERE`, `JOIN`, etc., so you can filter and join the data you're counting.

#### TODO
How many rows in the table `demographic_detail` represent an admission from the emergency room?

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_count_ed_admit

### Mins, Maxes, and Means
SQL has functions to calculate extreme values, means, and standard deviations. For example, the query below calculates the earliest/latest dates of birth and death:

In [ ]:
query = """
SELECT 
    MIN(dob) earliest_birth, MAX(dob) latest_birth, MIN(dod) earliest_death, MAX(dod) latest_death
FROM d_patients p
"""
pd.read_sql(query, conn)

We can also pass in transformed values to functions. Earlier we'd seen how to calculate how old a patient was in days or years. Now we can calculate summary statistics like the mean and standard deviation using `AVG()` and `STD()`:

In [ ]:
query = """
SELECT 
    AVG(DATEDIFF(dod, dob)) avg_age_at_death_days,
    STD(DATEDIFF(dod, dob)) std_age_at_death_days
FROM d_patients p
"""
pd.read_sql(query, conn)

#### TODO
Calculate the min, max, mean, and standard deviation age at death *in years*.

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_summary_stats_age_years

In [ ]:
query = """
SELECT 
    ____ min_age_at_death,
    MAX(____) ____,
    ____ mean_age_at_death,
    ____ sd_age_at_death
FROM d_patients p
"""
pd.read_sql(query, conn)

### GROUP BY
The queries above gave us single aggregate stats over an entire set of patients. But we might want to break our statistics up into groups. We'll use the `GROUP BY` clause for that. 

The `GROUP BY` clause tells us which column to use for breaking our patients up into groups. This works just like `df.groupby` in pandas. 

The query below counts the number of patients by `sex`:

In [ ]:
query = """
SELECT sex, COUNT(1) n
FROM d_patients 
GROUP BY sex
"""
pd.read_sql(query, conn)

#### TODO
Count the number of hospital admissions grouped by `admission_source_descr`.

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_count_admission_source

In [ ]:
query = """

"""
pd.read_sql(query, conn)

We can also group by multiple columns at once. This query calculates the number of patients in `demographic_detail` grouped by both sex and race and sorts in descending order of count.

In [ ]:
query = """
SELECT 
    d.ethnicity_descr, 
    p.sex,
    COUNT(*) n
FROM d_patients p
    INNER JOIN demographic_detail d
        ON p.subject_id = d.subject_id
GROUP BY d.ethnicity_descr, p.sex
ORDER BY COUNT(*) desc
LIMIT 10
"""
pd.read_sql(query, conn)

#### TODO
Write a query to count how many patients died in the hospital grouped by `sex`. Then answer the quiz below.

In [ ]:
# RUN CELL TO SEE HINT
hint_count_hospital_expire_by_sex

In [ ]:
# RUN CELL TO SEE QUIZ
quiz_count_hospital_expire_by_sex

In [ ]:
query = """

"""
pd.read_sql(query, conn)